[View in Colaboratory](https://colab.research.google.com/github/sanzgiri/orange_mamba_nbadh18/blob/master/NBA_Add_GoogleTrends.ipynb)

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

### **Run the below code **

!mkdir -p drive
!google-drive-ocamlfuse drive


Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

In [0]:
import pandas as pd
df1 = pd.read_csv("drive/NBA_Data_Hackathon/nba1617_gt.csv") 
df2 = pd.read_csv("drive/NBA_Data_Hackathon/nba1718_gt.csv") 

In [7]:
df1.head()

,Date,Start (ET),Home,Home_Points,Away,Away_Points,Attend.,GT_Home,GT_Away
0,2016-10-25,7:30 pm,Cleveland Cavaliers,117.0,New York Knicks,88.0,20562.0,99.0,51.0
1,2016-10-25,10:30 pm,Golden State Warriors,100.0,San Antonio Spurs,129.0,19596.0,100.0,100.0
2,2016-10-25,10:00 pm,Portland Trail Blazers,113.0,Utah Jazz,104.0,19446.0,12.0,18.0
3,2016-10-26,7:30 pm,Boston Celtics,122.0,Brooklyn Nets,117.0,18624.0,100.0,24.0
4,2016-10-26,7:00 pm,Indiana Pacers,130.0,Dallas Mavericks,121.0,17923.0,40.0,100.0


In [8]:
df2.head()


,Date,Start (ET),Home,Home_Points,Away,Away_Points,Attend.,GT_Home,GT_Away
0,2017-10-17,8:01 pm,Cleveland Cavaliers,102.0,Boston Celtics,99.0,20562.0,100.0,63.0
1,2017-10-17,10:30 pm,Golden State Warriors,121.0,Houston Rockets,122.0,19596.0,55.0,31.0
2,2017-10-18,7:30 pm,Boston Celtics,100.0,Milwaukee Bucks,108.0,18624.0,100.0,49.0
3,2017-10-18,8:30 pm,Dallas Mavericks,111.0,Atlanta Hawks,117.0,19709.0,80.0,100.0
4,2017-10-18,7:00 pm,Detroit Pistons,102.0,Charlotte Hornets,90.0,20491.0,48.0,30.0


In [9]:
df1.dtypes

Date            object
Start (ET)      object
Home            object
Home_Points    float64
Away            object
Away_Points    float64
Attend.        float64
GT_Home        float64
GT_Away        float64
dtype: object

In [19]:
#!pip install  pytrends
!pip install --upgrade git+https://github.com/GeneralMills/pytrends
!pip install lxml

  Cloning https://github.com/GeneralMills/pytrends to /tmp/pip-req-build-44k2_98t
  Running setup.py bdist_wheel for pytrends ... - done
  Stored in directory: /tmp/pip-ephem-wheel-cache-ykbaa_24/wheels/f6/49/f7/a4785ff2079f1cc793186a60d40863c5d4ee9863e2b315a0bd
Successfully built pytrends
  Found existing installation: pytrends 4.4.0
    Uninstalling pytrends-4.4.0:
      Successfully uninstalled pytrends-4.4.0


In [11]:
teams1 = list(df1.Home.unique())
teams2 = list(df2.Home.unique())
set(teams1) - set(teams2)

set()

In [10]:
print(teams1)

['Cleveland Cavaliers', 'Golden State Warriors', 'Portland Trail Blazers', 'Boston Celtics', 'Indiana Pacers', 'Los Angeles Lakers', 'Memphis Grizzlies', 'Milwaukee Bucks', 'New Orleans Pelicans', 'Orlando Magic', 'Philadelphia 76ers', 'Phoenix Suns', 'Toronto Raptors', 'Atlanta Hawks', 'Chicago Bulls', 'Sacramento Kings', 'Brooklyn Nets', 'Dallas Mavericks', 'Detroit Pistons', 'Miami Heat', 'Oklahoma City Thunder', 'Utah Jazz', 'Charlotte Hornets', 'Denver Nuggets', 'New York Knicks', 'San Antonio Spurs', 'Houston Rockets', 'Los Angeles Clippers', 'Minnesota Timberwolves', 'Washington Wizards']


In [0]:
season1 = pd.date_range("2016-10-01", "2017-06-30").to_series()
season2 = pd.date_range("2017-10-01", "2018-06-30").to_series()

In [0]:
from pytrends.request import TrendReq
import time

pytrends = TrendReq(hl='en-US')

from collections import defaultdict
team_gts_dict = defaultdict(dict)

for day in season1:
  
  tf = day.strftime('%Y-%m-%d %Y-%m-%d')
  print("Running for {0}".format(tf))
  nchunks = (len(teams1)/5) 
  
  for n in range(int(nchunks)):
    teamq = teams1[n * 5: n * 5 + 5]
    print(day, teamq)
    
    pytrends.build_payload(teamq, cat=264, timeframe=tf)
    time.sleep(1)
    dft = pytrends.interest_over_time().reset_index()
    
    for t in dft.columns.values:
      if t not in ['isPartial', 'date']:
        gts = dft[t].iloc[0]
        team_gts_dict[day][t] = gts
    

In [0]:
from pytrends.request import TrendReq
import time

pytrends = TrendReq(hl='en-US')


from collections import defaultdict
team_gts_dict2 = defaultdict(dict)

for day in season2:
  
  tf = day.strftime('%Y-%m-%d %Y-%m-%d')
  print("Running for {0}".format(tf))
  nchunks = (len(teams2)/5) 
  
  for n in range(int(nchunks)):
    teamq = teams2[n * 5: n * 5 + 5]
    print(day, teamq)
    
    pytrends.build_payload(teamq, cat=264, timeframe=tf)
    time.sleep(2)
    dft = pytrends.interest_over_time().reset_index()
    
    for t in dft.columns.values:
      if t not in ['isPartial', 'date']:
        gts = dft[t].iloc[0]
        team_gts_dict2[day][t] = gts

In [59]:
dfs1 = pd.DataFrame.from_dict(team_gts_dict)
dfs1 = dfs1.T.reset_index()
dfs1 = pd.melt(dfs1,["index"], var_name="team", value_name="gts")
dfs1 = dfs1.rename(columns={'index':'date'})
dfs1.head()
#dff2.to_csv("drive/NBA_Data_Hackathon/nba_gts.csv", index=None) 

,date,team,gts
0,2016-10-01,Atlanta Hawks,25.0
1,2016-10-02,Atlanta Hawks,22.0
2,2016-10-03,Atlanta Hawks,15.0
3,2016-10-04,Atlanta Hawks,10.0
4,2016-10-05,Atlanta Hawks,20.0


In [0]:
dff2 = pd.melt(dff,["index"], var_name="team", value_name="gts")
dff2 = dff2.rename(columns={'index':'date'})

In [0]:
dff2.to_csv("drive/NBA_Data_Hackathon/nba_gts.csv", index=None) 

In [2]:
!pip install --upgrade git+https://github.com/GeneralMills/pytrends
!pip install lxml

import pandas as pd
import sys
from pytrends.request import TrendReq
import time
from collections import defaultdict

def get_gts(start_date, end_date):

    season = pd.date_range(start_date, end_date).to_series()

    end_date_str = end_date.replace('-','')
    outfile = 'drive/NBA_Data_Hackathon/gts_{0}.csv'.format(end_date_str)
    #outfile = 'gts_{0}.csv'.format(end_date_str)

    pytrends = TrendReq(hl='en-US')

    team_gts_dict = defaultdict(dict)
    teams = ['Cleveland Cavaliers', 'Golden State Warriors', 'Portland Trail Blazers', 'Boston Celtics', 'Indiana Pacers',
             'Los Angeles Lakers', 'Memphis Grizzlies', 'Milwaukee Bucks', 'New Orleans Pelicans', 'Orlando Magic',
             'Philadelphia 76ers', 'Phoenix Suns', 'Toronto Raptors', 'Atlanta Hawks', 'Chicago Bulls',
             'Sacramento Kings', 'Brooklyn Nets', 'Dallas Mavericks', 'Detroit Pistons', 'Miami Heat',
             'Oklahoma City Thunder', 'Utah Jazz', 'Charlotte Hornets', 'Denver Nuggets', 'New York Knicks',
             'San Antonio Spurs', 'Houston Rockets', 'Los Angeles Clippers', 'Minnesota Timberwolves', 'Washington Wizards']

    for day in season:

        tf = day.strftime('%Y-%m-%d %Y-%m-%d')
        print("Running for {0}".format(tf))
        nchunks = (len(teams)/5)

        for n in range(int(nchunks)):
            teamq = teams[n * 5: n * 5 + 5]
            #print(day, teamq)

            pytrends.build_payload(teamq, cat=264, timeframe=tf)
            time.sleep(1)
            dft = pytrends.interest_over_time().reset_index()

            for t in dft.columns.values:
                if t not in ['isPartial', 'date']:
                    gts = dft[t].iloc[0]
                    team_gts_dict[day][t] = gts


    dfs = pd.DataFrame.from_dict(team_gts_dict)
    dfs = dfs.T.reset_index()
    dfs = pd.melt(dfs,["index"], var_name="team", value_name="gts")
    dfs = dfs.rename(columns={'index':'date'})
    dfs.to_csv(outfile, index=None)

  Cloning https://github.com/GeneralMills/pytrends to /tmp/pip-req-build-s2jxxqh8
    100% |████████████████████████████████| 5.8MB 5.2MB/s 
  Running setup.py bdist_wheel for pytrends ... - done
  Stored in directory: /tmp/pip-ephem-wheel-cache-qc3inqb6/wheels/f6/49/f7/a4785ff2079f1cc793186a60d40863c5d4ee9863e2b315a0bd
Successfully built pytrends


In [0]:
get_gts('2016-08-01', '2016-08-31')

In [0]:
get_gts('2016-09-01', '2016-09-30')
get_gts('2016-10-01', '2016-10-31')
get_gts('2016-11-01', '2016-11-30')
get_gts('2016-12-01', '2016-12-31')

In [0]:
#get_gts('2017-04-01', '2017-04-30')
#get_gts('2017-05-01', '2017-05-31')
#get_gts('2017-06-01', '2017-06-30')
#get_gts('2017-07-01', '2017-07-31')
#get_gts('2017-08-01', '2017-08-31')
#get_gts('2017-09-01', '2017-09-30')
#get_gts('2017-10-01', '2017-10-31')
#get_gts('2017-11-01', '2017-11-30')
get_gts('2017-12-01', '2017-12-31')

In [3]:
!ls

gts_20171231.csv  sample_data


In [0]:
from google.colab import files
!cp gts*.csv drive/NBA_Data_Hackathon

In [0]:
#get_gts('2018-01-01', '2018-01-31')
#get_gts('2018-02-01', '2018-02-28')
#get_gts('2018-03-01', '2018-03-31')
#get_gts('2018-04-01', '2018-04-30')
#get_gts('2018-05-01', '2018-05-31')
#get_gts('2018-06-01', '2018-06-30')
#get_gts('2018-07-01', '2018-07-31')
get_gts('2018-08-01', '2018-08-31')
get_gts('2018-09-01', '2018-09-09')

In [0]:
!cp gts*.csv drive/NBA_Data_Hackathon

In [3]:
get_gts('2018-08-01', '2018-08-31')
get_gts('2018-09-01', '2018-09-09')

Running for 2018-08-01 2018-08-01
Running for 2018-08-02 2018-08-02
Running for 2018-08-03 2018-08-03
Running for 2018-08-04 2018-08-04
Running for 2018-08-05 2018-08-05
Running for 2018-08-06 2018-08-06
Running for 2018-08-07 2018-08-07
Running for 2018-08-08 2018-08-08
Running for 2018-08-09 2018-08-09
Running for 2018-08-10 2018-08-10
Running for 2018-08-11 2018-08-11
Running for 2018-08-12 2018-08-12
Running for 2018-08-13 2018-08-13
Running for 2018-08-14 2018-08-14
Running for 2018-08-15 2018-08-15
Running for 2018-08-16 2018-08-16
Running for 2018-08-17 2018-08-17
Running for 2018-08-18 2018-08-18
Running for 2018-08-19 2018-08-19
Running for 2018-08-20 2018-08-20
Running for 2018-08-21 2018-08-21
Running for 2018-08-22 2018-08-22
Running for 2018-08-23 2018-08-23
Running for 2018-08-24 2018-08-24
Running for 2018-08-25 2018-08-25
Running for 2018-08-26 2018-08-26
Running for 2018-08-27 2018-08-27
Running for 2018-08-28 2018-08-28
Running for 2018-08-29 2018-08-29
Running for 20

IndexError: ignored

In [4]:
!cp gts*.csv drive/NBA_Data_Hackathon

cp: cannot stat 'gts*.csv': No such file or directory


In [0]:
import glob

In [0]:
datafiles = sorted(glob.glob('drive/NBA_Data_Hackathon/gts_*.csv'))

In [7]:
datafiles

['drive/NBA_Data_Hackathon/gts_20160131.csv',
 'drive/NBA_Data_Hackathon/gts_20160229.csv',
 'drive/NBA_Data_Hackathon/gts_20160331.csv',
 'drive/NBA_Data_Hackathon/gts_20160430.csv',
 'drive/NBA_Data_Hackathon/gts_20160531.csv',
 'drive/NBA_Data_Hackathon/gts_20160630.csv',
 'drive/NBA_Data_Hackathon/gts_20160731.csv',
 'drive/NBA_Data_Hackathon/gts_20160831.csv',
 'drive/NBA_Data_Hackathon/gts_20160930.csv',
 'drive/NBA_Data_Hackathon/gts_20161031.csv',
 'drive/NBA_Data_Hackathon/gts_20161130.csv',
 'drive/NBA_Data_Hackathon/gts_20161231.csv',
 'drive/NBA_Data_Hackathon/gts_20170131.csv',
 'drive/NBA_Data_Hackathon/gts_20170228.csv',
 'drive/NBA_Data_Hackathon/gts_20170331.csv',
 'drive/NBA_Data_Hackathon/gts_20170430.csv',
 'drive/NBA_Data_Hackathon/gts_20170531.csv',
 'drive/NBA_Data_Hackathon/gts_20170630.csv',
 'drive/NBA_Data_Hackathon/gts_20170731.csv',
 'drive/NBA_Data_Hackathon/gts_20170831.csv',
 'drive/NBA_Data_Hackathon/gts_20170930.csv',
 'drive/NBA_Data_Hackathon/gts_201

In [0]:
dfgts = pd.DataFrame()
for i in range(len(datafiles)):
  dftmp = pd.read_csv(datafiles[i])
  dfgts = dfgts.append(dftmp)

In [10]:
dfgts.shape

(29220, 3)

In [11]:
dfgts.head()

,date,team,gts
0,2016-01-01,Atlanta Hawks,24
1,2016-01-02,Atlanta Hawks,11
2,2016-01-03,Atlanta Hawks,24
3,2016-01-04,Atlanta Hawks,18
4,2016-01-05,Atlanta Hawks,31


In [0]:
dfgts_sorted = dfgts.sort_values(by=['date', 'team'], ascending=True).reset_index()

In [0]:
dfgts_sorted.to_csv('drive/NBA_Data_Hackathon/NBA_gts_161718.csv', index=None)